In [1]:
# # このセルは毎回実行しなくても大丈夫
# # Pipを最新バージョンにアップデート
# !wget https://bootstrap.pypa.io/get-pip.py
# !python get-pip.py
# # 必要なライブラリのインストール
# %pip install tokenizers fugashi ipadic accelerate==0.20.3 japanize-matplotlib
# # TransformersおよびDatasetsのインストール
# %pip install transformers datasets
# # データのダウンロード
# !wget https://github.com/ids-cv/wrime/raw/master/wrime-ver1.tsv
# # フォントのインストールとMatplotlibの設定
# %pip install matplotlib
# %pip install japanize-matplotlib
# %pip install scikit-learn

--2024-02-12 21:59:41--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2635835 (2.5M) [text/x-python]
Saving to: 'get-pip.py.1'

     0K .......... .......... .......... .......... ..........  1% 3.53M 1s
    50K .......... .......... .......... .......... ..........  3% 4.66M 1s
   100K .......... .......... .......... .......... ..........  5% 11.7M 0s
   150K .......... .......... .......... .......... ..........  7% 5.19M 0s
   200K .......... .......... .......... .......... ..........  9%  196M 0s
   250K .......... .......... .......... .......... .......... 11% 4.41M 0s
   300K .......... .......... .......... .......... .......... 13% 12.1M 0s
   350K .......... .......... .......... .......... .......... 15% 11.0M 0s
   400K .......... ..........

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


--2024-02-12 22:00:12--  https://github.com/ids-cv/wrime/raw/master/wrime-ver1.tsv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver1.tsv [following]
--2024-02-12 22:00:12--  https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver1.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9487235 (9.0M) [text/plain]
Saving to: 'wrime-ver1.tsv.1'

     0K .......... .......... .......... .......... ..........  0% 2.83M 3s
    50K .......... .......... .......... .......... ..........  1% 7.64M 2s
   100K .......... .......... .......... .......... ..........  1% 4.76

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from datasets import Dataset, load_metric

c:\Users\minaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
checkpoint = 'C:/Users/minaa/NewTCA/my_project/0212_this_is_it.bin'
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# モデルの作成
num_labels = 8
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)

emotion_names_jp = ['喜び', '悲しみ', '期待', '驚き', '怒り', '恐れ', '嫌悪', '信頼']

SafetensorError: Error while deserializing header: MetadataIncompleteBuffer

In [ ]:
def np_softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

In [ ]:
def analyze_emotion(text):
    # 推論モード
    model.eval()

    # 入力データ変換 + 推論
    tokens = tokenizer(text, truncation=True, return_tensors="pt")
    tokens.to(model.device)
    preds = model(**tokens)
    prob = np_softmax(preds.logits.cpu().detach().numpy()[0])
    out_dict = {n: p for n, p in zip(emotion_names_jp, prob)}
    out_list = list(out_dict.values())
    return out_list

In [12]:
directory_path = r'C:\Users\minaa\NewTCA\0212_omg_its_working_finally\data'

# ファイル名のリスト
file_names = ['2012-2014.csv', '2015-2017.csv', '2018-2020.csv', '2021-2023.csv']

dfs = {}

for file_name in file_names:
    file_path = os.path.join(directory_path, file_name)
    df = pd.read_csv(file_path)
    dfs[file_name] = df
    

In [13]:
def preprocess_dataframe(df):
    # 欠損値が全ての列を削除
    df = df.dropna(axis=1, how='all')
    
    # 'テキスト'列に欠損値がある行を削除
    df.dropna(subset=['テキスト'], inplace=True)
    
    # '@', '名前'列を基準に重複する行を削除
    df = df.drop_duplicates(subset=['@', '名前'], keep=False)
    
    # 特定のキーワードを含む'自己紹介'列を持つ行を削除
    exclude_keywords = ['党', '議員', '支持者', '副業', '儲かる', 'セミナー', '稼ぎたい', '相互フォロー', '友だち登録', '投資', 'ワクチン', '幸せ', '未成年', '成人', '占星術', '天皇', 'LINE', '万円', '闇']
    df = df[~df['自己紹介'].str.contains('|'.join(exclude_keywords), na=False, regex=True)]
    
    # インデックスをリセット
    df = df.reset_index(drop=True)
    
    return df

In [14]:
# 各DataFrameに対して前処理を行う
for key, df in dfs.items():
    # # 分析に不要なカラムを削除
    # columns_to_drop = ['投稿日時', 'URL', 'imp', 'リプライ数', 'RT数', 'いいね数', '引用数', '投稿アプリ', 'フォロー数', 'フォロワー数', 'ツイート数', 'アカウント作成日']
    # df.drop(columns=columns_to_drop, inplace=True)
    # ポスト選別
    dfs[key] = preprocess_dataframe(df)

C:\Users\minaa\AppData\Local\Temp\ipykernel_15444\561880786.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['テキスト'], inplace=True)
C:\Users\minaa\AppData\Local\Temp\ipykernel_15444\561880786.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['テキスト'], inplace=True)
C:\Users\minaa\AppData\Local\Temp\ipykernel_15444\561880786.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['テキ

In [15]:
def process_sentiment_data(file_name):
    sentiment_data = pd.DataFrame(columns=['テキスト', '喜び', '悲しみ', '期待', '驚き', '怒り', '恐れ', '嫌悪', '信頼'])

    df = dfs[file_name]
    corpus_list = df['テキスト'].to_list()

    data_to_append = []

    for text2 in corpus_list:
        model.eval()
        tokens = tokenizer(text2, truncation=True, return_tensors="pt")
        tokens.to(model.device)
        preds = model(**tokens)
        prob = np_softmax(preds.logits.cpu().detach().numpy()[0])

        new_row = pd.DataFrame({'テキスト': [text2], '喜び': [prob[0]], '悲しみ': [prob[1]],
                                '期待': [prob[2]], '驚き': [prob[3]], '怒り': [prob[4]],
                                '恐れ': [prob[5]], '嫌悪': [prob[6]], '信頼': [prob[7]]})

        data_to_append.append(new_row)

    sentiment_data = pd.concat(data_to_append, ignore_index=True)
    return sentiment_data

In [52]:
results = {}
for file_name in file_names:
    sentiment_df = process_sentiment_data(file_name)
    results[file_name] = sentiment_df


In [60]:
def update_dataframe(file_name):
    df = results[file_name]
    # データフレームの処理
    for index, row in df.iterrows():
        # テキストカラムを除外して最大値を探索する
        max_column = row.drop('テキスト').idxmax()
        # 各カラムの値を0に設定
        df.loc[index, ~df.columns.isin(['テキスト'])] = 0
        # 最大値のカラムの値を1に設定
        df.at[index, max_column] = 1
    
    return df

In [63]:
for file_name in file_names:
    results[file_name] = update_dataframe(file_name)
    

In [89]:
all_df = []

for file_name in file_names:
    df = results[file_name]
    df_without_text = df.drop(columns=['テキスト'])
    column_sums = df_without_text.sum()
    column_sums_df = pd.DataFrame(column_sums).T
    all_df.append(column_sums_df)

In [90]:
# all_dfに含まれるデータフレームを縦方向に連結
final_df = pd.concat(all_df, axis=0, ignore_index=True)

# データフレームを表示
print(final_df)

     喜び  悲しみ     期待    驚き   怒り    恐れ    嫌悪   信頼
0  25.0  5.0  126.0  32.0  0.0  41.0  21.0  0.0
1  12.0  6.0   83.0  25.0  0.0  33.0  23.0  0.0
2  20.0  5.0  106.0  25.0  0.0  37.0  13.0  0.0
3  27.0  8.0  120.0  20.0  0.0  26.0  17.0  0.0


In [88]:
file_names_df = pd.DataFrame(file_names, columns=['file_name'])

,file_name
0,2012-2014.csv
1,2015-2017.csv
2,2018-2020.csv
3,2021-2023.csv


In [92]:
# file_names_dfとfinal_dfを横方向に連結
merged_df = pd.concat([file_names_df, final_df], axis=1)
merged_df['合計'] = merged_df.iloc[:, 1:].sum(axis=1)

,file_name,喜び,悲しみ,期待,驚き,怒り,恐れ,嫌悪,信頼,合計
0,2012-2014.csv,25.0,5.0,126.0,32.0,0.0,41.0,21.0,0.0,250.0
1,2015-2017.csv,12.0,6.0,83.0,25.0,0.0,33.0,23.0,0.0,182.0
2,2018-2020.csv,20.0,5.0,106.0,25.0,0.0,37.0,13.0,0.0,206.0
3,2021-2023.csv,27.0,8.0,120.0,20.0,0.0,26.0,17.0,0.0,218.0


In [93]:
# 各列の値を合計で割る
for column in merged_df.columns[1:-1]:  # 'file_name'と'合計'列は除外する
    merged_df[column] = merged_df[column] / merged_df['合計']

In [95]:
# '合計'列の名前を 'サンプル数' に変更
merged_df.rename(columns={'合計': 'サンプル数'}, inplace=True)


In [96]:
# データフレームを表示
merged_df

,file_name,喜び,悲しみ,期待,驚き,怒り,恐れ,嫌悪,信頼,サンプル数
0,2012-2014.csv,0.100000,0.020000,0.504000,0.128000,0.0,0.164000,0.084000,0.0,250.0
1,2015-2017.csv,0.065934,0.032967,0.456044,0.137363,0.0,0.181319,0.126374,0.0,182.0
2,2018-2020.csv,0.097087,0.024272,0.514563,0.121359,0.0,0.179612,0.063107,0.0,206.0
3,2021-2023.csv,0.123853,0.036697,0.550459,0.091743,0.0,0.119266,0.077982,0.0,218.0


一番高いのはどの時代も期待
しいて言うなら驚きが減ってる？？